In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, expr, count
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, IntegerType, ArrayType, DateType
import sys
import os
from pyspark.sql import DataFrame


In [0]:
path_countries_silver = '/FileStore/silver/departments/countries'
df_countries = spark.read.format('parquet')\
.load(path_countries_silver)

In [0]:
path_regions_silver = '/FileStore/silver/departments/regions'
df_regions = spark.read.format('parquet')\
.load(path_regions_silver)

In [0]:
df_join = df_regions.join(df_countries,'id_regiao','inner').drop(df_countries.id_regiao)

In [0]:
df_join.display()

id_regiao,nome_regiao,id_pais,nome_pais
2,Americas,AR,Argentina
3,Asia,AU,Australia
1,Europe,BE,Belgium
2,Americas,BR,Brazil
2,Americas,CA,Canada
1,Europe,CH,Switzerland
3,Asia,CN,China
1,Europe,DE,Germany
1,Europe,DK,Denmark
4,Middle East and Africa,EG,Egypt


In [0]:
df_report = df_join.groupBy('nome_regiao').agg(
    count('id_pais').alias('qtd_paises_regiao')
).select('nome_regiao','qtd_paises_regiao')

In [0]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS department;
""")

Out[38]: DataFrame[]

In [0]:
path_countries_gold = '/FileStore/gold/departments/countries'
df_report.write.option('path', path_countries_gold).format('parquet').mode('overwrite').saveAsTable("department.countries_per_regions")

In [0]:
spark.sql("SELECT * FROM department.countries_per_regions").display()

nome_regiao,qtd_paises_regiao
Europe,8
Middle East and Africa,6
Americas,5
Asia,6
